# Extract Ortofotos as High Quality Images

This notebook extracts ortofotos from Comune di Bologna and saves them as high-quality PNG images.

## Setup

Install required packages:
```bash
pip install folium pillow requests langchain-google-community python-dotenv
```

**Note:** This notebook uses a direct tile download approach which is much more reliable than browser screenshots. No Selenium or ChromeDriver required!


In [13]:
import folium
import tempfile
from pathlib import Path
import time
from PIL import Image, ImageDraw
import io
import math
import requests
from io import BytesIO
from langchain_google_community.geocoding import GoogleGeocodingTool

# Optional: for Selenium approach (less reliable)
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service


## Configuration

Set the location (Bologna centro by default) and the years you want to extract.


In [14]:
from dotenv import load_dotenv

load_dotenv()

True

In [15]:
# Default location: Bologna centro (Piazza Maggiore)
DEFAULT_LAT = 44.4939
DEFAULT_LON = 11.3426

BBOX=None

# You can specify a custom location here
location = "Giardini Margherita"
if location:
    query = location
    google_geocoding_tool = GoogleGeocodingTool()
    result = google_geocoding_tool.run(query)
    # coordinates of the center of the bbox
    lat, lon = result[0]['geometry']['location']['lat'], result[0]['geometry']['location']['lng']
    # construct bbox from lat and lon in a square of side 1000 meters
    BBOX = [lon - 0.0025, lat - 0.0025, lon + 0.0025, lat + 0.0025]


# Zoom level (higher = more detail, 16-18 recommended)
ZOOM_LEVEL = 18

# Output directory
OUTPUT_DIR = Path("ortofoto_images")
OUTPUT_DIR.mkdir(exist_ok=True)

# Image size (width x height in pixels)
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080


## Alternative Approach: Direct Tile Download

This approach downloads tiles directly and stitches them together, which is more reliable than browser screenshots.


In [16]:
def deg2num(lat_deg, lon_deg, zoom):
    """Convert lat/lon to tile numbers"""
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return xtile, ytile

def num2deg(xtile, ytile, zoom):
    """Convert tile numbers to lat/lon"""
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return lat_deg, lon_deg

def download_tile(year, zoom, x, y, session=None):
    """Download a single tile from the Bologna server"""
    if session is None:
        session = requests.Session()
    
    # Use HTTP as in the original script
    url = f"http://sitmappe.comune.bologna.it/tms/tileserver/Ortofoto{year}/{zoom}/{x}/{y}.png"
    
    try:
        response = session.get(url, timeout=10)
        if response.status_code == 200:
            return Image.open(BytesIO(response.content))
        else:
            print(f"Warning: Failed to download tile {x},{y} (status {response.status_code})")
            return None
    except Exception as e:
        print(f"Warning: Error downloading tile {x},{y}: {e}")
        return None

def create_ortofoto_image(year, lat, lon, zoom=ZOOM_LEVEL, tile_radius=3):
    """
    Create an ortofoto image by downloading and stitching tiles.
    
    Args:
        year: Year of the ortofoto
        lat: Latitude of the center
        lon: Longitude of the center
        zoom: Zoom level (higher = more detail)
        tile_radius: Number of tiles in each direction from center
    
    Returns:
        PIL Image object
    """
    # Get the center tile
    center_x, center_y = deg2num(lat, lon, zoom)
    
    # Calculate tile range
    x_min = center_x - tile_radius
    x_max = center_x + tile_radius
    y_min = center_y - tile_radius
    y_max = center_y + tile_radius
    
    # Tile size (standard)
    TILE_SIZE = 256
    
    # Calculate image size
    width = (x_max - x_min + 1) * TILE_SIZE
    height = (y_max - y_min + 1) * TILE_SIZE
    
    # Create blank image
    result_image = Image.new('RGB', (width, height), color=(200, 200, 200))
    
    # Download and paste tiles
    session = requests.Session()
    total_tiles = (x_max - x_min + 1) * (y_max - y_min + 1)
    current_tile = 0
    
    print(f"Downloading {total_tiles} tiles for year {year}...")
    
    for x in range(x_min, x_max + 1):
        for y in range(y_min, y_max + 1):
            current_tile += 1
            if current_tile % 10 == 0:
                print(f"  Progress: {current_tile}/{total_tiles} tiles")
            
            tile_img = download_tile(year, zoom, x, y, session)
            
            if tile_img:
                # Calculate position in result image
                pos_x = (x - x_min) * TILE_SIZE
                pos_y = (y - y_min) * TILE_SIZE
                result_image.paste(tile_img, (pos_x, pos_y))
    
    print(f"✓ Downloaded all tiles for {year}")
    return result_image


## Extract Ortofoto Using Direct Download (Recommended)

This method is more reliable than Selenium screenshots.


In [17]:
# Extract single ortofoto for 2024
year = 2024

# Use the center coordinates from your configuration
if BBOX:
    lat = (BBOX[1] + BBOX[3]) / 2
    lon = (BBOX[0] + BBOX[2]) / 2
else:
    lat = DEFAULT_LAT
    lon = DEFAULT_LON

print(f"Extracting ortofoto for {year} at location ({lat:.4f}, {lon:.4f})...")

# Create the image by downloading tiles
# tile_radius controls the area: 3 = 7x7 tiles = 1792x1792 pixels
# Increase for larger area, decrease for smaller
ortofoto_img = create_ortofoto_image(
    year=year,
    lat=lat,
    lon=lon,
    zoom=ZOOM_LEVEL,
    tile_radius=4  # 9x9 tiles = 2304x2304 pixels
)

# Save the image
output_file = OUTPUT_DIR / f"ortofoto_{year}_direct.png"
ortofoto_img.save(output_file, 'PNG', optimize=True, quality=95)

print(f"\n✓ Saved high-quality image to: {output_file}")
print(f"  Image size: {ortofoto_img.size[0]}x{ortofoto_img.size[1]} pixels")


Extracting ortofoto for 2024 at location (44.4827, 11.3524)...
  Progress: 10/81 tiles
  Progress: 20/81 tiles
  Progress: 30/81 tiles
  Progress: 40/81 tiles
  Progress: 50/81 tiles
  Progress: 60/81 tiles
  Progress: 70/81 tiles
  Progress: 80/81 tiles
✓ Downloaded all tiles for 2024

✓ Saved high-quality image to: ortofoto_images/ortofoto_2024_direct.png
  Image size: 2304x2304 pixels


## Extract Two Ortofotos for Comparison (2017 and 2024)

Download ortofotos from 2017 and 2024 for the same location.


In [18]:
# Years to compare
years = [2017, 2024]

# Use the center coordinates from your configuration
if BBOX:
    lat = (BBOX[1] + BBOX[3]) / 2
    lon = (BBOX[0] + BBOX[2]) / 2
else:
    lat = DEFAULT_LAT
    lon = DEFAULT_LON

print(f"Extracting ortofotos at location ({lat:.4f}, {lon:.4f})...\n")

images = {}
for year in years:
    print(f"Processing year {year}...")
    
    # Create the image by downloading tiles
    ortofoto_img = create_ortofoto_image(
        year=year,
        lat=lat,
        lon=lon,
        zoom=ZOOM_LEVEL,
        tile_radius=4  # 9x9 tiles for good coverage
    )
    
    # Save the image
    output_file = OUTPUT_DIR / f"ortofoto_{year}_direct.png"
    ortofoto_img.save(output_file, 'PNG', optimize=True, quality=95)
    
    print(f"✓ Saved: {output_file}")
    print(f"  Size: {ortofoto_img.size[0]}x{ortofoto_img.size[1]} pixels\n")
    
    images[year] = ortofoto_img

print("="*50)
print("All ortofotos extracted successfully!")
print(f"Images saved in: {OUTPUT_DIR}")
print("="*50)


Extracting ortofotos at location (44.4827, 11.3524)...

Processing year 2017...
  Progress: 10/81 tiles
  Progress: 20/81 tiles
  Progress: 30/81 tiles
  Progress: 40/81 tiles
  Progress: 50/81 tiles
  Progress: 60/81 tiles
  Progress: 70/81 tiles
  Progress: 80/81 tiles
✓ Downloaded all tiles for 2017
✓ Saved: ortofoto_images/ortofoto_2017_direct.png
  Size: 2304x2304 pixels

Processing year 2024...
  Progress: 10/81 tiles
  Progress: 20/81 tiles
  Progress: 30/81 tiles
  Progress: 40/81 tiles
  Progress: 50/81 tiles
  Progress: 60/81 tiles
  Progress: 70/81 tiles
  Progress: 80/81 tiles
✓ Downloaded all tiles for 2024
✓ Saved: ortofoto_images/ortofoto_2024_direct.png
  Size: 2304x2304 pixels

All ortofotos extracted successfully!
Images saved in: ortofoto_images


## Create Side-by-Side Comparison

Combine the 2017 and 2024 images into a single comparison image.


In [19]:
# Load the images if not already in memory
if 'images' not in locals():
    img_2017 = Image.open(OUTPUT_DIR / "ortofoto_2017_direct.png")
    img_2024 = Image.open(OUTPUT_DIR / "ortofoto_2024_direct.png")
else:
    img_2017 = images[2017]
    img_2024 = images[2024]

# Ensure both images have the same size
if img_2017.size != img_2024.size:
    print("Warning: Images have different sizes. Resizing to match...")
    target_size = (min(img_2017.width, img_2024.width), min(img_2017.height, img_2024.height))
    img_2017 = img_2017.crop((0, 0, target_size[0], target_size[1]))
    img_2024 = img_2024.crop((0, 0, target_size[0], target_size[1]))

# Create a new image with both side by side
width, height = img_2017.size
comparison = Image.new('RGB', (width * 2 + 10, height))  # +10 for separator

# Add a white separator line
from PIL import ImageDraw
comparison.paste((255, 255, 255), (0, 0, width * 2 + 10, height))

# Paste the images
comparison.paste(img_2017, (0, 0))
comparison.paste(img_2024, (width + 10, 0))

# Draw a separator line
draw = ImageDraw.Draw(comparison)
draw.line([(width + 5, 0), (width + 5, height)], fill=(0, 0, 0), width=2)

# Save comparison image
comparison_path = OUTPUT_DIR / "ortofoto_comparison_2017_2024.png"
comparison.save(comparison_path, 'PNG', optimize=True, quality=95)

print(f"✓ Side-by-side comparison saved to: {comparison_path}")
print(f"  Image size: {comparison.size[0]}x{comparison.size[1]} pixels")


✓ Side-by-side comparison saved to: ortofoto_images/ortofoto_comparison_2017_2024.png
  Image size: 4618x2304 pixels


## Available Years

The following ortofoto years are available from Comune di Bologna:
- 2017
- 2018
- 2020
- 2021
- 2022
- 2023
- 2024

You can modify the `years` list in the cells above to extract different years.
